In [1]:
import os
import sys
import pandas as pd 
import numpy as np 
import scipy.sparse as ssparse 
import scipy.io as sio 

sys.path.insert(0, '/Users/bingyu')
from sp import interface 

In [2]:
nodes_df0 = pd.read_csv('../projects/berkeley/network_inputs/osm_nodes.csv')
links_df0 = pd.read_csv('../projects/berkeley/network_inputs/osm_edges.csv')
# links_df0 = pd.read_csv('../projects/berkeley/network_inputs/modified_network_edges.csv')
wgh = links_df0['length']
row = links_df0['start_igraph']
col = links_df0['end_igraph']
assert max(np.max(row)+1, np.max(col)+1) == nodes_df0.shape[0], 'nodes and links dimension do not match, row {}, col {}, nodes {}'.format(np.max(row), np.max(col), nodes_df0.shape[0])
g_coo = ssparse.coo_matrix((wgh, (row, col)), shape=(nodes_df0.shape[0], nodes_df0.shape[0]))
print("({}, {}), {}".format(g_coo.shape[0], g_coo.shape[1], len(g_coo.data)))
sio.mmwrite('network_sparse.mtx', g_coo)
# g_coo = sio.mmread(absolute_path+'/outputs/network_sparse.mtx'.format(folder))
g = interface.readgraph(bytes('network_sparse.mtx', encoding='utf-8'))

(15294, 15294), 37951


In [5]:
### marin: 10185, 2598
### spruce: 2160, 1626
### shasta: 5525, 1623
### oxfful: 14299, 9242
### rose: 4864, 14299
### cedar: 15068, 14483
### university: 1775, 8104
### shamlk: 5541, 8084
counterflow_start_igraph, counterflow_end_igraph = 5541, 8084 ### node_id_igraph
counterflow_start_sp = counterflow_start_igraph + 1
counterflow_end_sp = counterflow_end_igraph + 1

### along counterflow
sp_1 = g.dijkstra(counterflow_start_sp, counterflow_end_sp)
sp_dist_1 = sp_1.distance(counterflow_end_sp)
print(sp_dist_1)
sp_route_1 = sp_1.route(counterflow_end_sp)
path_1 = [(start_sp-1, end_sp-1) for (start_sp, end_sp) in sp_route_1]
print(path_1)
path_df_1 = pd.DataFrame(path_1, columns=['start_igraph', 'end_igraph'])
path_df_1['along'] = 1

### against counterflow
if counterflow_end_igraph == 8104:
    counterflow_end_igraph = 568
    counterflow_end_sp = counterflow_end_igraph + 1
sp_2 = g.dijkstra(counterflow_end_sp, counterflow_start_sp)
sp_dist_2 = sp_2.distance(counterflow_start_sp)
print(sp_dist_2)
sp_route_2 = sp_2.route(counterflow_start_sp)
path_2 = [(start_sp-1, end_sp-1) for (start_sp, end_sp) in sp_route_2]
print(path_2)
path_df_2 = pd.DataFrame(path_2, columns=['start_igraph', 'end_igraph'])
path_df_2['along'] = 0

counterflow_route = pd.concat([path_df_1, path_df_2])
counterflow_route = pd.merge(counterflow_route, links_df0[['start_igraph', 'end_igraph', 'edge_id_igraph','geometry']], how='left', on=['start_igraph', 'end_igraph'])
counterflow_route.to_csv('shamlk.csv', index=False)

4278.350224634792
[(5541, 13983), (13983, 5540), (5540, 14835), (14835, 12105), (12105, 12110), (12110, 12103), (12103, 12102), (12102, 12109), (12109, 12100), (12100, 12099), (12099, 875), (875, 8204), (8204, 8202), (8202, 8199), (8199, 12095), (12095, 2154), (2154, 1404), (1404, 3217), (3217, 3212), (3212, 3214), (3214, 7614), (7614, 3211), (3211, 14674), (14674, 14643), (14643, 5431), (5431, 3234), (3234, 3235), (3235, 920), (920, 3219), (3219, 3228), (3228, 3210), (3210, 3207), (3207, 3224), (3224, 1003), (1003, 5204), (5204, 671), (671, 673), (673, 676), (676, 683), (683, 686), (686, 689), (689, 9532), (9532, 700), (700, 702), (702, 704), (704, 12908), (12908, 711), (711, 710), (710, 721), (721, 8084)]
4326.149204185702
[(8084, 712), (712, 2757), (2757, 2758), (2758, 12909), (12909, 4434), (4434, 3806), (3806, 4349), (4349, 13673), (13673, 13503), (13503, 888), (888, 13164), (13164, 6370), (6370, 11058), (11058, 758), (758, 13502), (13502, 13915), (13915, 2664), (2664, 2644), (264